# Chargement des données de pandémies - Partie 3: Vérification des données

Ce notebook réalise la vérification des données chargées dans la base de données MySQL.

## 1. Configuration initiale

Importation des bibliothèques nécessaires et définition des paramètres de connexion à la base de données.

In [ ]:
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Configuration de la visualisation
%matplotlib inline
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Définition des chemins vers les fichiers sources
chemin_base = Path().resolve().parent.parent if 'notebooks' in str(Path().resolve()) else Path().resolve().parent
dossier_donnees_transformees = chemin_base / "donnees" / "transformees"

# Affichage du chemin
print(f"Dossier des données transformées: {dossier_donnees_transformees}")

## 2. Configuration de la connexion à la base de données

Définition des paramètres de connexion à la base de données MySQL.

In [ ]:
# Paramètres de connexion à la base de données
config_db = {
    "host": "localhost",
    "user": "root",
    "password": "",  # Laisser vide si aucun mot de passe
    "database": "epiviz",
    "port": 3306
}

print("Paramètres de connexion à la base de données:")
for cle, valeur in config_db.items():
    if cle != "password":
        print(f"- {cle}: {valeur}")
    else:
        print(f"- {cle}: {'(aucun)' if valeur == '' else '******'}")

## 3. Fonction de connexion à la base de données

Définition d'une fonction pour établir une connexion à la base de données MySQL.

In [ ]:
def connecter_mysql(config=None):
    """
    Établit une connexion à la base de données MySQL.
    
    Args:
        config (dict): Dictionnaire contenant les paramètres de connexion.
        
    Returns:
        tuple: (connexion, curseur) si la connexion est établie, (None, None) sinon.
    """
    if config is None:
        config = config_db.copy()
    
    try:
        # Connexion à la base de données
        connexion = mysql.connector.connect(**config)
        curseur = connexion.cursor()
        print(f"Connexion à la base de données '{config['database']}' établie avec succès.")
        return connexion, curseur
    
    except Error as e:
        print(f"Erreur lors de la connexion à la base de données: {e}")
        return None, None

## 4. Fonction d'exécution de requêtes SQL

Définition d'une fonction pour exécuter des requêtes SQL et récupérer les résultats sous forme de DataFrame.

In [ ]:
def executer_requete(connexion, curseur, requete):
    """
    Exécute une requête SQL et récupère les résultats sous forme de DataFrame.
    
    Args:
        connexion: Connexion à la base de données.
        curseur: Curseur de la base de données.
        requete (str): Requête SQL à exécuter.
        
    Returns:
        DataFrame: Résultats de la requête sous forme de DataFrame.
    """
    try:
        # Exécution de la requête
        curseur.execute(requete)
        
        # Récupération des résultats
        resultats = curseur.fetchall()
        
        # Récupération des noms des colonnes
        colonnes = [i[0] for i in curseur.description]
        
        # Création du DataFrame
        df = pd.DataFrame(resultats, columns=colonnes)
        
        return df
    
    except Error as e:
        print(f"Erreur lors de l'exécution de la requête: {e}")
        return None

## 5. Vérification du nombre de lignes dans chaque table

Vérification du nombre de lignes dans chaque table de la base de données.

In [ ]:
# Vérification du nombre de lignes dans chaque table
print("Vérification du nombre de lignes dans chaque table...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération de la liste des tables
        curseur.execute("SHOW TABLES;")
        tables = [table[0] for table in curseur.fetchall()]
        
        # Vérification du nombre de lignes dans chaque table
        for table in tables:
            requete = f"SELECT COUNT(*) AS nombre_lignes FROM {table};"
            df = executer_requete(connexion, curseur, requete)
            if df is not None:
                print(f"Table {table}: {df.iloc[0]['nombre_lignes']} lignes")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification du nombre de lignes: {e}")

## 6. Vérification des données de la table calendrier

Vérification des données de la table calendrier.

In [ ]:
# Vérification des données de la table calendrier
print("Vérification des données de la table calendrier...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération des données de la table calendrier
        requete = "SELECT * FROM calendrier ORDER BY date LIMIT 10;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nAperçu des données de la table calendrier:")
            display(df)
        
        # Vérification de la plage de dates
        requete = "SELECT MIN(date) AS date_min, MAX(date) AS date_max FROM calendrier;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nPlage de dates dans la table calendrier:")
            print(f"Date minimale: {df.iloc[0]['date_min']}")
            print(f"Date maximale: {df.iloc[0]['date_max']}")
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des données de la table calendrier: {e}")

## 7. Vérification des données de la table localisation

Vérification des données de la table localisation.

In [ ]:
# Vérification des données de la table localisation
print("Vérification des données de la table localisation...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération des données de la table localisation
        requete = "SELECT * FROM localisation ORDER BY pays LIMIT 10;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nAperçu des données de la table localisation:")
            display(df)
        
        # Vérification du nombre de pays par continent
        requete = "SELECT continent, COUNT(*) AS nombre_pays FROM localisation GROUP BY continent ORDER BY nombre_pays DESC;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nNombre de pays par continent:")
            display(df)
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des données de la table localisation: {e}")

## 8. Vérification des données de la table pandemie

Vérification des données de la table pandemie.

In [ ]:
# Vérification des données de la table pandemie
print("Vérification des données de la table pandemie...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération des données de la table pandemie
        requete = "SELECT * FROM pandemie;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nDonnées de la table pandemie:")
            display(df)
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des données de la table pandemie: {e}")

## 9. Vérification des données de la table donnees

Vérification des données de la table donnees.

In [ ]:
# Vérification des données de la table donnees
print("Vérification des données de la table donnees...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Récupération des données de la table donnees
        requete = "SELECT * FROM donnees LIMIT 10;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nAperçu des données de la table donnees:")
            display(df)
        
        # Vérification des statistiques descriptives
        requete = "SELECT MIN(cas_confirmes) AS min_cas, MAX(cas_confirmes) AS max_cas, AVG(cas_confirmes) AS avg_cas, MIN(deces) AS min_deces, MAX(deces) AS max_deces, AVG(deces) AS avg_deces FROM donnees;"
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nStatistiques descriptives de la table donnees:")
            display(df)
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de la vérification des données de la table donnees: {e}")

## 10. Requêtes d'analyse sur les données

Exécution de requêtes d'analyse sur les données chargées.

In [ ]:
# Requêtes d'analyse sur les données
print("Requêtes d'analyse sur les données...")

try:
    # Connexion à la base de données
    connexion, curseur = connecter_mysql()
    
    if connexion is not None and curseur is not None:
        # Requête 1: Nombre total de cas confirmés et de décès par pandémie
        requete = """
        SELECT p.nom_pandemie, SUM(d.cas_confirmes) AS total_cas, SUM(d.deces) AS total_deces
        FROM donnees d
        JOIN pandemie p ON d.id_pandemie = p.id_pandemie
        GROUP BY p.nom_pandemie
        ORDER BY total_cas DESC;
        """
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nNombre total de cas confirmés et de décès par pandémie:")
            display(df)
            
            # Visualisation des résultats
            plt.figure(figsize=(12, 6))
            
            # Graphique des cas confirmés
            plt.subplot(1, 2, 1)
            sns.barplot(x="nom_pandemie", y="total_cas", data=df)
            plt.title("Nombre total de cas confirmés par pandémie")
            plt.xlabel("Pandémie")
            plt.ylabel("Nombre de cas confirmés")
            plt.xticks(rotation=45)
            
            # Graphique des décès
            plt.subplot(1, 2, 2)
            sns.barplot(x="nom_pandemie", y="total_deces", data=df)
            plt.title("Nombre total de décès par pandémie")
            plt.xlabel("Pandémie")
            plt.ylabel("Nombre de décès")
            plt.xticks(rotation=45)
            
            plt.tight_layout()
            plt.show()
        
        # Requête 2: Top 10 des pays les plus touchés par le COVID-19
        requete = """
        SELECT l.pays, SUM(d.cas_confirmes) AS total_cas, SUM(d.deces) AS total_deces
        FROM donnees d
        JOIN localisation l ON d.id_localisation = l.id_localisation
        JOIN pandemie p ON d.id_pandemie = p.id_pandemie
        WHERE p.nom_pandemie = 'COVID-19'
        GROUP BY l.pays
        ORDER BY total_cas DESC
        LIMIT 10;
        """
        df = executer_requete(connexion, curseur, requete)
        if df is not None:
            print("\nTop 10 des pays les plus touchés par le COVID-19:")
            display(df)
            
            # Visualisation des résultats
            plt.figure(figsize=(14, 6))
            
            # Graphique des cas confirmés
            plt.subplot(1, 2, 1)
            sns.barplot(x="total_cas", y="pays", data=df.sort_values("total_cas"))
            plt.title("Top 10 des pays les plus touchés par le COVID-19 (cas confirmés)")
            plt.xlabel("Nombre de cas confirmés")
            plt.ylabel("Pays")
            
            # Graphique des décès
            plt.subplot(1, 2, 2)
            sns.barplot(x="total_deces", y="pays", data=df.sort_values("total_deces"))
            plt.title("Top 10 des pays les plus touchés par le COVID-19 (décès)")
            plt.xlabel("Nombre de décès")
            plt.ylabel("Pays")
            
            plt.tight_layout()
            plt.show()
        
        # Requête 3: Évolution des cas de COVID-19 au fil du temps pour un pays spécifique (États-Unis)
        requete = """
        SELECT c.date, d.cas_confirmes, d.deces
        FROM donnees d
        JOIN calendrier c ON d.id_date = c.id_date
        JOIN localisation l ON d.id_localisation = l.id_localisation
        JOIN pandemie p ON d.id_pandemie = p.id_pandemie
        WHERE p.nom_pandemie = 'COVID-19' AND l.pays = 'US'
        ORDER BY c.date;
        """
        df = executer_requete(connexion, curseur, requete)
        if df is not None and len(df) > 0:
            print("\nÉvolution des cas de COVID-19 aux États-Unis:")
            display(df.head())
            
            # Conversion de la colonne date en datetime
            df["date"] = pd.to_datetime(df["date"])
            
            # Visualisation des résultats
            plt.figure(figsize=(14, 6))
            
            # Graphique des cas confirmés
            plt.subplot(1, 2, 1)
            plt.plot(df["date"], df["cas_confirmes"], marker="o", linestyle="-", markersize=2)
            plt.title("Évolution des cas de COVID-19 aux États-Unis")
            plt.xlabel("Date")
            plt.ylabel("Nombre de cas confirmés")
            plt.xticks(rotation=45)
            plt.grid(True)
            
            # Graphique des décès
            plt.subplot(1, 2, 2)
            plt.plot(df["date"], df["deces"], marker="o", linestyle="-", color="red", markersize=2)
            plt.title("Évolution des décès dus au COVID-19 aux États-Unis")
            plt.xlabel("Date")
            plt.ylabel("Nombre de décès")
            plt.xticks(rotation=45)
            plt.grid(True)
            
            plt.tight_layout()
            plt.show()
        
        # Fermeture de la connexion
        curseur.close()
        connexion.close()
        print("\nConnexion fermée.")
    else:
        print("Impossible d'établir une connexion à la base de données.")
except Exception as e:
    print(f"Erreur lors de l'exécution des requêtes d'analyse: {e}")

## Conclusion

Dans ce notebook, nous avons réalisé la vérification des données chargées dans la base de données MySQL. Les principales opérations ont été :

1. Vérification du nombre de lignes dans chaque table
2. Vérification des données de la table calendrier
3. Vérification des données de la table localisation
4. Vérification des données de la table pandemie
5. Vérification des données de la table donnees
6. Exécution de requêtes d'analyse sur les données chargées

Les données sont maintenant chargées dans la base de données MySQL et ont été vérifiées. Elles sont prêtes à être utilisées pour l'analyse et la visualisation.

Le processus ETL (Extraction, Transformation, Chargement) est maintenant complet, avec les données extraites des fichiers CSV sources, transformées pour correspondre au schéma de la base de données, et chargées dans la base de données MySQL.